In [20]:
import pandas as pd
import numpy as np

In [21]:
df = pd.read_csv("DonneesMeteo.txt",delim_whitespace=True, low_memory=False, skiprows=1)

In [22]:
import datetime
#time = temp = pd.DatetimeIndex(df1['DT_local'])
date = pd.DatetimeIndex(pd.to_datetime(df['Date'], format="%Y%m%d"))
df['MONTH'] = date.month
df['DAY'] = date.day
df['YEAR'] = date.year
df['DOY'] = date.dayofyear
#Version arrondie à la minute de la ligne du dessous : num = pd.to_datetime(pd.Series.round(df['hh(UTC)']*3600)*1000000000)
hh = pd.DatetimeIndex(pd.to_datetime(df['hh(UTC)']*3600*1000000000))
df['HOUR'] = hh.hour
df['MINUTE'] = hh.minute

On va à présent se concentrer sur la prédiction de :
    - puissance
Selon
    - T
    - vent
    - skycover
    - precipitation

In [23]:
#df.columns.values.tolist()

# RandomForest

In [44]:
#Paramètres
'''colsX = ['DAY',
    'Date',
 'hh(UTC)',
 'Tp1(C)',
 'Tp2(C)',
 'IrrPOA(W/m2)',
 'Gz2(W/m2)',
 'Drz2(W/m2)',
 'Dfz2(W/m2)',
 'IRz2(W/m2)',
 'T(C)',
 'WS(m/s)',
 'WD(deg)',
 'mpGz1(W/m2)',
 'IRz1dn(W/m2)',
 'IRz1up(W/m2)',
 'mpT(C)',
 'mpWS(m/s)',
 'mpWD(deg)',
 'SZA(deg)',
 'SAA(deg)']'''
colsX = ["DOY", "hh(UTC)", "IrrPOA(W/m2)"]
colsY = ["P1(W)", "P2(W)"]
#print(cols)

In [45]:
import sklearn.cross_validation as cv
import sklearn.preprocessing as pp #pour les scaling

In [46]:
def prepare(df,colsX,colsY):
    #Split the treated data between X and y
    Z = vireNan(df[colsX+colsY])
    deb = 0
    sep = len(colsX)#separteur entre les X et les Y
    fin = sep+len(colsY)
    X=Z[:,deb:sep]
    y=Z[:,sep:fin]
    return X,y

import numpy as np
def vireNan(Z) :
    # Load the dataset
    Z = np.array(Z).astype(np.float)

    #Remove nan and infinite values
    masknan = ~np.any(np.isnan(Z), axis=1)
    Z = Z[masknan]
    maskfin = np.any(np.isfinite(Z), axis=1)
    Z = Z[maskfin]
    return Z

In [47]:
#preparer (nan, découpe)
X,y = prepare(df,colsX,colsY)

In [48]:
def decoupe(X,y) :
    X_train, X_1, y_train, y_1 = cv.train_test_split(X, y, test_size=0.4, random_state=0)
    X_cv, X_test, y_cv, y_test = cv.train_test_split(X_1, y_1, test_size=0.1, random_state=0)
    return X_train, X_cv, X_test, y_train, y_cv, y_test

In [49]:
#Découper
X_train, X_cv, X_test, y_train, y_cv, y_test = decoupe(X,y)

In [50]:
#variable globales
fsX = pp.StandardScaler()
fsy = pp.StandardScaler()
def featureScale(X_train, X_cv, X_test, y_train, y_cv, y_test) :
    #feature scaling inti
    fsX.fit(X_train)
    X_train = fsX.transform(X_train)
    X_cv = fsX.transform(X_cv)
    X_test = fsX.transform(X_test)
    
    fsy.fit(y_train)
    y_train = fsy.transform(y_train)
    y_cv = fsy.transform(y_cv)
    y_test = fsy.transform(y_test)
    
    return X_train, X_cv, X_test, y_train, y_cv, y_test

In [51]:
#scaler
X_train, X_cv, X_test, y_train, y_cv, y_test = featureScale(X_train, X_cv, X_test, y_train, y_cv, y_test)


In [42]:
from sklearn.ensemble import RandomForestRegressor

def rms(Y1,Y2):
    return np.sqrt(np.mean((Y1 - Y2)**2))
    
def randomforestcross(nArray, X_train, X_cv, y_train, y_cv):
    #Conversion
    nArray = np.array(nArray).astype(np.integer)
    #tableau avec les valeurs d'erreur
    RMS=np.zeros((nArray.size,2))
    
    i=0
    for n in nArray:
        y_=randomforest(n,X_train,y_train,X_cv)
        r =rms(y_,y_cv)
        RMS[i,0]=n
        RMS[i,1]=r
        i+=1
    plt.scatter(RMS[:,0],RMS[:,1])
    plt.show()
    
    return RMS

def randomforest(n,X,Y,X_test):
    clf= RandomForestRegressor(n_estimators=n)
    clf.fit(X,Y)
    y_=clf.predict(X_test)
    return y_

def gradientcross(X,Y):
    X_train, X_cv, X_test, y_train, y_cv, y_test = decoupe(X,y)
    RMS=np.zeros((250,2))
    
    for i in range(1,10):
        i = i * 10 + 100
        y_=gradient(i,X_train,y_train,X_cv)
        r=rms(y_,y_cv)
        RMS[i,0]=i
        RMS[i,1]=r
        
    plt.scatter(RMS[:,0],RMS[:,1])
    plt.show()
    L = [X_train, y_train, X_test, y_test]
    return L 

def gradient(n,X,Y,X_t):
    clf= ensemble.GradientBoostingRegressor(n_estimators=n)
    clf.fit(X,Y)
    y_=clf.predict(X_t)
    return y_

In [52]:
#Pour plusieurs k ?
nArray = [1,3,10,30,100]
#Cross sur les nArray
RMS = randomforestcross(nArray, X_train, X_cv, y_train, y_cv)

In [43]:
plotterIterations(X_test,1,0,y_test,0)

[-1.69632413 -1.57997273 -1.46362133 -1.34726993 -1.23091853 -1.11456712
 -0.99821572 -0.88186432 -0.76551292 -0.64916152 -0.53281012 -0.41645872
 -0.30010732 -0.18375591 -0.06740451  0.04894689  0.16529829  0.28164969
  0.39800109  0.51435249  0.63070389  0.7470553   0.8634067   0.9797581
  1.0961095   1.2124609   1.3288123   1.4451637   1.5615151   1.6778665 ]


In [7]:
# Plot outputs 2D
import matplotlib.cm as cm

def plotterIterations(X,nX,nIt,y,ny) :
    #On se ramène à des journées
    EltIteration = X[:,nIt]
    Iterations = np.unique(EltIteration)
    print(Iterations)

    #Gérer plusieurs couleurs
    colors = cm.rainbow(np.linspace(0, 1, len(Iterations)))

    for k,c in zip(Iterations,colors)  :
        #On prend que le jour
        condition = EltIteration == k #entre 60 et 153
        xPlot = np.extract(condition, X[:,nX])
        yPlot = np.extract(condition, y[:,ny])
        plt.scatter(xPlot,yPlot, color=c,s=2)
        #sort
        #print(np.concatenate(xPlot,yPlot))
        #z = np.sort(np.concatenate(xPlot,yPlot),0)
        #plt.plot(z[:,0], z[:,1], color=c,linewidth=2)
        #print(k)
        #print(xPlot)
        #print(yPlot)
        #print(z[:,0])
        #print(z[:,1])
        #break

    #plt.plot(X_test, regr.predict(X_test), color='blue',linewidth=3)
    plt.xticks(())
    plt.yticks(())

    plt.title('')

    plt.show()

In [18]:
import sklearn.cross_validation as cv
import sklearn.preprocessing as pp #pour les scaling
from sklearn.ensemble import RandomForestRegressor

def rms(Y1,Y2):
    return np.sqrt(np.mean((Y1 - Y2)**2))
    
def randomforestcross(nArray, X_train, X_cv, y_train, y_cv):
    #tableau avec les valeurs d'erreur
    RMS=np.zeros((nArray.size,2))
    
    i=0
    for n in nArray:
        y_=randomforest(n,X_train,y_train,X_cv)
        r =rms(y_,y_cv)
        RMS[i,0]=n
        RMS[i,1]=r
        i+=1
    plt.scatter(RMS[:,0],RMS[:,1])
    plt.show()
    
    return RMS

def randomforest(n,X,Y,X_test):
    clf= RandomForestRegressor(n_estimators=n)
    clf.fit(X,Y)
    y_=clf.predict(X_test)
    return y_

def gradientcross(X,Y):
    X_train, X_cv, X_test, y_train, y_cv, y_test = decoupe(X,y)
    RMS=np.zeros((250,2))
    
    for i in range(1,10):
        i = i * 10 + 100
        y_=gradient(i,X_train,y_train,X_cv)
        r=rms(y_,y_cv)
        RMS[i,0]=i
        RMS[i,1]=r
    plt.scatter(RMS[:,0],RMS[:,1])
    plt.show()
    L = [X_train, y_train, X_test, y_test]
    return L 
def gradient(n,X,Y,X_t):
    clf= ensemble.GradientBoostingRegressor(n_estimators=n)
    clf.fit(X,Y)
    y_=clf.predict(X_t)
    return y_


def prepare(df,colsX,colsY):
    #Split the treated data between X and y
    Z = vireNan(df[colsX+colsY])
    deb = 0
    sep = len(colsX)#separteur entre les X et les Y
    fin = sep+len(colsY)
    X=Z[:,deb:sep]
    y=Z[:,sep:fin]
    return X,y

import numpy as np
def vireNan(Z) :
    # Load the dataset
    Z = np.array(Z).astype(np.float)

    #Remove nan and infinite values
    masknan = ~np.any(np.isnan(Z), axis=1)
    Z = Z[masknan]
    maskfin = np.any(np.isfinite(Z), axis=1)
    Z = Z[maskfin]
    return Z

def decoupe(X,y) :
    X_train, X_1, y_train, y_1 = cv.train_test_split(X, y, test_size=0.4, random_state=0)
    X_cv, X_test, y_cv, y_test = cv.train_test_split(X_1, y_1, test_size=0.1, random_state=0)
    return X_train, X_cv, X_test, y_train, y_cv, y_test

#variable globales
fsX = pp.StandardScaler()
fsy = pp.StandardScaler()
def featureScale(X_train, X_cv, X_test, y_train, y_cv, y_test) :
    #feature scaling inti
    fsX.fit(X_train)
    X_train = fsX.transform(X_train)
    X_cv = fsX.transform(X_cv)
    X_test = fsX.transform(X_test)
    
    fsy.fit(y_train)
    y_train = fsy.transform(y_train)
    y_cv = fsy.transform(y_cv)
    y_test = fsy.transform(y_test)
    
    return X_train, X_cv, X_test, y_train, y_cv, y_test
